In [27]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns = 50
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score,accuracy_score
from sklearn.metrics import roc_curve
from sklearn.preprocessing import StandardScaler

In [28]:
data = pd.read_table('bank-additional-full.csv',delimiter=';')
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [29]:
data['job'] = data['job'].replace(['admin.', 'management'], 'white-collar')
data['job'] = data['job'].replace(['housemaid', 'services'], 'pink-collar')
data['job'] = data['job'].replace(['retired', 'student', 'unemployed', 'unknown'], 'others')
data['education'] = data['education'].replace(['basic.4y', 'basic.6y','basic.9y'], 'school')
data['education'] = data['education'].replace(['professional.course', 'university.degree'], 'university')
data['education'] = data['education'].replace(['illiterate', 'unknown'], 'others')
data['education'] = data['education'].replace('high.school', 'high school')
data['age'] = np.log(data['age'])

data['total'] = data['campaign'] + data['previous']
del data['campaign']
del data['previous']

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['y'] = le.fit_transform(data['y'])
data['contact'] = le.fit_transform(data['contact'])
data['month'] = le.fit_transform(data['month'])

data['month'] = data['month'].replace([0,1,3,4,6,7],1)
data['month'] = data['month'].replace([2,5,8,9],0)
np.unique(data['month'])

data = pd.get_dummies(data=data, columns = ['job' ,'marital', 'education', 'poutcome','loan','housing','default','day_of_week'], \
                                   prefix = ['job','marital', 'education', 'poutcome','loan','housing','default','day_of_week'])

In [30]:
cols = data.select_dtypes(include=['int','float']).columns.values
stand_scale= StandardScaler() 
data.loc[:,cols] = stand_scale.fit_transform(data.loc[:,cols])

In [31]:
data.head()

,age,contact,month,duration,pdays,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,total,job_blue-collar,job_entrepreneur,job_others,job_pink-collar,job_self-employed,job_technician,job_white-collar,marital_divorced,marital_married,marital_single,marital_unknown,education_high school,education_others,education_school,education_university,poutcome_failure,poutcome_nonexistent,poutcome_success,loan_no,loan_unknown,loan_yes,housing_no,housing_unknown,housing_yes,default_no,default_unknown,default_yes,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed
0,1.450950,1,1,261,999,0.648092,0.722722,0.886447,0.71246,0.33168,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0
1,1.520664,1,1,149,999,0.648092,0.722722,0.886447,0.71246,0.33168,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0
2,-0.181403,1,1,226,999,0.648092,0.722722,0.886447,0.71246,0.33168,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0
3,0.125669,1,1,151,999,0.648092,0.722722,0.886447,0.71246,0.33168,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0
4,1.450950,1,1,307,999,0.648092,0.722722,0.886447,0.71246,0.33168,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0


In [32]:
y = data.y
X = data.drop('y', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=9, test_size=0.2)
X_train_a, X_val, y_train_a, y_val = train_test_split(X_train,y_train,random_state=9, test_size=0.2)
X_a, X_b, y_a, y_b = train_test_split(X_train_a,y_train_a,random_state=9, test_size=0.2)

In [34]:
# predicting for min_samples 0f leaf as 2.
for min_smpl_split in [20, 40, 50,100,200]:
    for estim in [5,10,50,100,200]:
        clf = RandomForestClassifier(min_samples_split=min_smpl_split,n_estimators=estim, random_state=9)
        clf.fit(X_a,y_a)
        y_pred = clf.predict_proba(X_b)
        y_pred = [p[1] for p in y_pred]
        #accuracy_clf_1 = accuracy_score(y_pred, y_val)
        print (roc_auc_score(y_b, y_pred), min_smpl_split,estim)

(0.9234185577465327, 20, 5)
(0.93226840425012791, 20, 10)
(0.93886909822192188, 20, 50)
(0.94008870393410893, 20, 100)
(0.94098047640206051, 20, 200)
(0.92729825413662448, 40, 5)
(0.93368559049059785, 40, 10)
(0.93949100907696326, 40, 50)
(0.93964712888157687, 40, 100)
(0.94104138330585807, 40, 200)
(0.92831037277593487, 50, 5)
(0.93099669745155977, 50, 10)
(0.93821031300644908, 50, 50)
(0.93987571322534713, 50, 100)
(0.94061320043398, 50, 200)
(0.92309255904759224, 100, 5)
(0.92926213487503628, 100, 10)
(0.93902888711712584, 100, 50)
(0.93954145907257858, 100, 100)
(0.94037067354596693, 100, 200)
(0.93241241605579361, 200, 5)
(0.93251441677420155, 200, 10)
(0.93725928472546571, 200, 50)
(0.93754437306432536, 200, 100)
(0.93777956177115818, 200, 200)


In [35]:
clf_1 = RandomForestClassifier(min_samples_split=20,n_estimators=200,max_depth=5, random_state=9)
clf_1.fit(X_a,y_a)
lst = []
for est in clf_1.estimators_:
    lst.append(est.predict_proba(X_b)[:,1])
    
df = pd.DataFrame(lst)
df = df.transpose()
#df['y_train_b'] = y_train_b
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,...,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199
0,0.052343,0.050535,0.042700,0.039339,0.059528,0.050408,0.010389,0.099339,0.033333,0.006866,0.018349,0.048277,0.050334,0.051141,0.049651,0.021976,0.027022,0.068961,0.012636,0.004944,0.062627,0.050466,0.008600,0.049039,0.052055,...,0.047905,0.030201,0.015387,0.015711,0.064711,0.060146,0.048899,0.018411,0.060005,0.010455,0.011676,0.048175,0.011166,0.038634,0.008188,0.013797,0.057666,0.061761,0.036681,0.053598,0.041899,0.05250,0.089439,0.040068,0.061224
1,0.052343,0.050535,0.063790,0.079119,0.059528,0.050408,0.067172,0.064213,0.056093,0.006866,0.031651,0.048277,0.050334,0.051141,0.078243,0.021976,0.027022,0.013738,0.055407,0.010451,0.061624,0.050466,0.034599,0.178197,0.057909,...,0.047905,0.030201,0.015387,0.015711,0.105904,0.060146,0.056739,0.018411,0.024708,0.010455,0.059041,0.048175,0.011166,0.071615,0.008188,0.013797,0.057666,0.061761,0.057984,0.053598,0.048361,0.05250,0.011598,0.054480,0.063208
2,0.029188,0.027034,0.031258,0.024046,0.029421,0.050408,0.010389,0.064213,0.056093,0.006866,0.018349,0.048277,0.032450,0.044309,0.031375,0.012234,0.027022,0.013738,0.016211,0.004944,0.054783,0.032799,0.008600,0.049039,0.057909,...,0.047905,0.030201,0.015387,0.015711,0.008110,0.029796,0.040506,0.018411,0.060005,0.010455,0.011676,0.030151,0.011166,0.038634,0.008188,0.013797,0.044627,0.061761,0.031610,0.027572,0.033048,0.01066,0.011598,0.040068,0.041022
3,0.052343,0.050535,0.042700,0.039339,0.077922,0.050408,0.010389,0.064213,0.056093,0.006866,0.018349,0.048277,0.050334,0.051141,0.049651,0.021976,0.027022,0.013738,0.016211,0.004944,0.061624,0.054280,0.008600,0.049039,0.057909,...,0.047905,0.030201,0.015387,0.015711,0.008110,0.060146,0.040506,0.018411,0.024708,0.010455,0.011676,0.046729,0.011166,0.038634,0.008188,0.013797,0.028881,0.061761,0.036681,0.028999,0.048361,0.05250,0.011598,0.040068,0.041022
4,0.052343,0.050535,0.055295,0.079119,0.013718,0.050408,0.067172,0.064213,0.056093,0.006866,0.031651,0.048277,0.060899,0.051141,0.049651,0.021976,0.027022,0.013738,0.016211,0.010451,0.054783,0.050466,0.034599,0.009772,0.057909,...,0.047905,0.030201,0.015387,0.015711,0.064711,0.060146,0.056739,0.018411,0.060005,0.010455,0.059041,0.069712,0.011166,0.071615,0.008188,0.013797,0.057666,0.061761,0.057984,0.053598,0.048875,0.01066,0.011598,0.054480,0.049383


In [23]:
lst2 = []
for est1 in clf_1.estimators_:
    lst2.append(est1.predict_proba(X_test)[:,1])
df2 = pd.DataFrame(lst2)
df2 = df2.transpose()
df2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,...,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,0.083598,0.044575,0.170455,0.304348,0.190994,0.135338,0.015845,0.110000,0.293103,0.184524,0.037736,0.433333,0.281250,0.088400,0.028692,0.233221,0.020806,0.014950,0.071506,0.086129,0.091684,0.093750,0.233129,0.084625,0.265823,...,0.243902,0.298701,0.280899,0.131783,0.029412,0.100070,0.257669,0.145161,0.519231,0.202381,0.160000,0.110764,0.169197,0.421053,0.227642,0.032307,0.026554,0.184864,0.113178,0.019505,0.492063,0.172285,0.288889,0.150000,0.102549
1,0.411290,0.452381,0.338983,0.360759,0.532258,0.138889,0.846154,0.101449,0.611940,0.916667,0.394366,0.103448,0.267490,0.446809,0.250000,0.092593,0.375000,0.424242,0.448980,0.279070,0.475610,0.204819,0.084337,0.574297,0.529412,...,0.689655,0.512195,0.911111,0.538462,0.571429,0.507177,0.381818,0.299435,0.881720,0.239193,0.732394,0.703704,1.000000,0.457983,0.089286,0.476190,0.273973,0.547739,0.289062,0.883721,0.258427,0.734694,0.237500,0.765625,0.500000
2,0.800000,0.312500,0.833333,0.666667,0.000000,0.519231,0.492537,0.476821,0.050000,0.462500,1.000000,0.871429,0.603774,0.684211,0.369231,0.775510,0.562500,0.472222,0.384615,0.312500,0.166667,0.882353,0.538462,0.583333,0.407407,...,0.255952,0.225352,0.580645,0.690141,0.809524,0.652174,0.487805,0.571429,0.315068,0.066667,0.633333,1.000000,0.000000,0.365079,0.551282,0.458333,0.328767,0.813953,0.666667,0.524194,0.282051,0.250000,0.458333,0.312500,0.625000
3,0.088889,0.054178,0.000000,0.002764,0.000000,0.003887,0.013844,0.005312,0.004572,0.065943,0.002588,0.000000,0.027778,0.000000,0.054145,0.013551,0.004020,0.003708,0.004777,0.020892,0.051306,0.005988,0.003559,0.055135,0.048679,...,0.006105,0.002553,0.000000,0.007883,0.010029,0.062347,0.064147,0.074556,0.001170,0.100629,0.004566,0.097514,0.001873,0.003333,0.002123,0.057192,0.001638,0.000000,0.067073,0.008044,0.000570,0.003014,0.015873,0.008513,0.073454
4,0.263158,0.119048,0.103074,0.062363,0.051685,0.049505,0.169421,0.036704,0.206388,0.169811,0.101852,0.097561,0.127907,0.088400,0.020408,0.146608,0.072000,0.069707,0.050000,0.227723,0.170543,0.035526,0.217391,0.206030,0.100000,...,0.006105,0.192771,0.056590,0.133333,0.115942,0.136872,0.113793,0.155716,0.107843,0.077840,0.060531,0.215686,0.076923,0.205240,0.236220,0.138122,0.153040,0.071342,0.081886,0.035148,0.153153,0.131944,0.154519,0.149718,0.024590


In [24]:
from sklearn.linear_model import LogisticRegression

lg = LogisticRegression(random_state=9)
lg.fit(df,y_val)
y_pred = lg.predict(df2)
print roc_auc_score(y_test, y_pred)

0.74288131986
